<h1 align= "center"> <strong> Training Pipeline Configurations </strong> </h1>
<center>---------------------------- </center>

# Table of Contents
* [1. Writing JSON to a file in python](#1.-Writing-JSON-to-a-file-in-python)
* [2. Reading JSON from a file using python](#2.-Reading-JSON-from-a-file-using-python)

## 1. Writing JSON to a file in python

In [11]:
#define the project details in here
project_name = "your_projectname"
BUName = "your_BUName"
job_name='tr'
env = 'dev'
#define your tags in here
ProjectName = "ProjectName"
DepartmentName = "DepartmentName"
OwnerName = "OwnerName"
#Once again please add your input datasets name here
input1 = "your_first_input_dataset.csv"
input2 = "your_second_input_dataset.csv"
input3 = "your_third_input_dataset.csv"

If you need to change the insatnce type and instance count, please follow the comments in the below cell of the script.

In [12]:
import json
import pytz
from datetime import datetime
import boto3
srilanka_tz = pytz.timezone('Asia/Colombo')
s3 = boto3.client('s3')
date_folder = datetime.now(srilanka_tz).strftime("%Y-%m-%d")
  
# Data to be written
dictionary ={
    "pipeline": {
        "pipeline_config" : {
            "model_package_group_name": f"{BUName}{project_name}-ModelPackageGroup",
            "pipeline_name": f"{BUName}-{project_name}-{job_name}-pipeline-",
            "base_job_prefix": f"{BUName}-{project_name}",
            "env": f"{env}",
            "project_name": f"{project_name}"
        },
        "generic_tags": {
            "ProjectName": f"{ProjectName}", 
            "DepartmentName": f"{DepartmentName}",
            "UsecaseName": f"{project_name}",
            "ResourceName": "sagemaker_notebook",
            "OwnerName": f"{OwnerName}",
            "BUName": f"{BUName}"
        }
    },
    "tr_preprocessing": {
        "estimator_config": {
            "ecr_repository": f"{BUName}-{project_name}-preprocessing-image",
            "instance_count": 1, #change the instance count
            "instance_type": "ml.m5.large" #change the instance type
        },
        "inputs": {
            "code" : f"{BUName}_{project_name}_{job_name}_preprocessing/preprocessing.py",
            "input1": f"s3://dlk-cloud-tier-10-preprocessed-ml-dev/{project_name}/{job_name}/sample_input1_dataset/{input1}",
            "input2": f"s3://dlk-cloud-tier-10-preprocessed-ml-dev/{project_name}/{job_name}/sample_input2_dataset/{input2}",
            "input3": f"s3://dlk-cloud-tier-10-preprocessed-ml-dev/{project_name}/{job_name}/sample_input3_dataset/{input3}"
        },
    },
    "train_model": {
        "estimator_config": {
            "ecr_repository": f"{BUName}-{project_name}-trainingdt-image",
            "instance_count": 2, #change the instance type
            "instance_type": "ml.m5.4xlarge", #change the instance count
        },
    },
    "register_model": {
        "estimator_config": {
            "transform_instances": "ml.m5.xlarge", #change the instance type
            "inference_instances": "ml.m5.xlarge" #change the instance type
        },
    },

}

In [13]:
import json
import boto3    
s3 = boto3.resource('s3')
s3object = s3.Object('dlk-cloud-tier-8-code-ml-dev', f'config_files/pipeline_config/{BUName}_{project_name}/{date_folder}/pipeline_config.json')

s3object.put(
    Body=(bytes(json.dumps(dictionary).encode('UTF-8')))
)

{'ResponseMetadata': {'RequestId': 'K5EDT908MWY1M4Z7',
  'HostId': '05K11sjtyWACyqCc5i34og+Vmatug2zMYxVbuokYf19BQoKSl9uO7C1u/+LJJVABl1KhH/zgd3A=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '05K11sjtyWACyqCc5i34og+Vmatug2zMYxVbuokYf19BQoKSl9uO7C1u/+LJJVABl1KhH/zgd3A=',
   'x-amz-request-id': 'K5EDT908MWY1M4Z7',
   'date': 'Mon, 14 Mar 2022 09:13:50 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"1ae0937859c6e816b048621ab09eca3a"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"1ae0937859c6e816b048621ab09eca3a"',
 'ServerSideEncryption': 'AES256'}

## 2. Reading JSON from a file using python

In [1]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

In [2]:
env = 'dev'

In [4]:
import json
import pytz
from datetime import datetime
import boto3

###################################### configuration file management ###############################


print("setup the date folder to do the data versioning")
srilanka_tz = pytz.timezone('Asia/Colombo')
s3 = boto3.client('s3')
date_folder = datetime.now(srilanka_tz).strftime("%Y-%m-%d")


print("setting up the stage configuration file into this file")

print("pick the latest trained config file")
def latest_config(config_type):
    '''
    this will pick the latest trained config file
    '''

    s3_path = f"s3://dlk-cloud-tier-8-code-ml-{env}/"
    config_bucket = f"dlk-cloud-tier-8-code-ml-{env}"
    s3 = boto3.client('s3')
    all_objects = s3.list_objects(Bucket = f"dlk-cloud-tier-8-code-ml-{env}", Prefix=f"config_files/{config_type}/{BUName}_{project_name}/")
    model_folder=all_objects['Contents'][-1]['Key']
    #model_path=s3_path+model_folder
    
    return config_bucket, model_folder

print("calling the latest stage config file")
config_type = "pipeline_config"
config_bucket = latest_config(config_type)[0]
model_path = latest_config(config_type)[1]

print("loading the stage config data from s3")

print("import the getJsonData function")
def getJsonData(bucket_name,key_name):
    '''
    this will pick the json config file from s3 bucket
    '''
    
    print("[LOG]", bucket_name,'---------')
    print("[LOG]", key_name,'--------------')
      
    s3_client = boto3.client('s3')
    csv_obj = s3_client.get_object(Bucket=bucket_name, Key=key_name)
    
    body = csv_obj['Body']
    
    json_string = body.read().decode('utf-8')
    json_content = json.loads(json_string)
    
    return json_content


config = getJsonData(config_bucket, model_path)


print("json script loaded successfully")
#########################################################################################


setup the date folder to do the data versioning
setting up the stage configuration file into this file
enter your project name with BU Name here.
pick the latest trained config file
calling the latest stage config file
loading the stage config data from s3
import the getJsonData function
[LOG] dlk-cloud-tier-8-code-ml-dev ---------
[LOG] config_files/pipeline_config/mobile_prepaidchurn/2022-03-14/pipeline_config.json --------------
json script loaded successfully


In [11]:
config_bucket = ParameterString(
        name="ConfigBucket",
        default_value=f"dlk-cloud-tier-8-code-ml-{env}",  # Change this to point to the s3 location of your raw input data.
    )
config_s3_prefix = ParameterString(
        name="ConfigS3Prefix",
        default_value=f'config_files/pipeline_config/{BUName}_{project_name}/{date_folder}/pipeline_config.json',  # Change this to point to the s3 location of your raw input data.
    )    
    
config = getJsonData(config_bucket,config_s3_prefix)

[LOG] dlk-cloud-tier-8-code-ml-dev ---------
[LOG] config_files/pipeline_config/mobile_prepaidchurn/pipeline_config.json --------------


### Experiments

In [6]:
config

{'pipeline': {'pipeline_config': {'model_package_group_name': 'MobilePrepaidchurn-ModelPackageGroup',
   'pipeline_name': 'mobile-prepaidchurn-tr-pipeline-',
   'base_job_prefix': 'mobile-prepaidchurn',
   'env': 'dev',
   'project_name': 'prepaidchurn'},
  'inputs': {'input1': 's3://dlk-cloud-tier-10-preprocessed-ml-dev/mobile_prepaidchurn/sample_dataset/Sample AWS.csv',
   'input2': 'enter the second s3 input file location.csv'},
  'generic_tags': {'ProjectName': 'mlops_migration',
   'DepartmentName': 'dialog_ml',
   'UsecaseName': 'prepaidchurn',
   'ResourceName': 'sagemaker_notebook',
   'OwnerName': 'mlops',
   'BUName': 'mobile'}},
 'tr_preprocessing': {'estimator_config': {'ecr_repository': 'mobile-prepaidchurn-preprocessing-image',
   'instance_count': 1,
   'instance_type': 'ml.m5.large'},
  'inputs': {'code': 'mobile_prepaidchurn_tr_preprocessing/preprocessing.py'}},
 'train_model_dt': {'estimator_config': {'ecr_repository': 'mobile-prepaidchurn-trainingdt-image',
   'insta

In [9]:
config['register_model_dt']['estimator_config']['transform_instances']

'ml.m5.xlarge'

In [10]:
config['pipeline']['inputs']['input1']

's3://dlk-cloud-tier-10-preprocessed-ml-dev/mobile_prepaidchurn/sample_dataset/Sample AWS.csv'

In [21]:
config['tr_preprocessing']['inputs']['code']

'mobile_prepaidchurn_tr_preprocessing/preprocessing_config.py'

In [12]:
base_job_prefix=config['pipeline']['pipeline_config']['base_job_prefix']

In [13]:
f"{base_job_prefix}-preprocessing"

'mobile-prepaidchurn-preprocessing'

In [13]:
config['pipeline']['pipeline_config']['model_package_group_name']

'AWSWorkShop-ModelPackageGroup'

In [13]:
config['pipeline']['pipeline_config']['pipeline_name']

'SalesStockPred-pipeline-'

In [14]:
config['pipeline']['pipeline_config']['base_job_prefix']

'SalesStockPred'

In [15]:
config['pipeline']['pipeline_config']['env']

'dev'

In [16]:
config['pipeline']['pipeline_config']['project_name']

'sales_stockpred'

In [18]:
config['pipeline']['inputs']['input1']

's3://dlk-cloud-tier-10-preprocessed-ml-dev/sales_stockpred/temp_data/gc_retailers_3yr/temp/'

In [19]:
config['pipeline']['inputs']['input2']

's3://dlk-cloud-tier-10-preprocessed-ml-dev/sales_stockpred/temp_data/gc_retailers_3yr/gc_retailers_2022_JAN10.csv'

In [24]:
config['pipeline']['generic_tags']['UsecaseName']

'sales-stockpred'

In [28]:
config['preprocessing']['estimator_config']

{'ecr_repository': 'sales-stockpred-preprocessing-image',
 'instance_count': 1,
 'instance_type': 'ml.m5.large'}

In [33]:
config['train_model']['estimator_config']['instance_count']

2